In [1]:
working_directory = '/mnt/DataRAID/melismail/PDAC'
import os
os.chdir(working_directory)
from pickle_utils import write_pickle, read_pickle

import sys, cv2, math
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir 
from os.path import isfile, join
from numpy import argmax
from tifffile import imread, imsave
from glob import glob

import sklearn
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, preprocessing
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report
from scikitplot.metrics import plot_roc, plot_precision_recall, plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

np.random.seed(109)

In [2]:
base_path = '/mnt/DataRAID/melismail/PDAC/data'
preprocessing_path ='Preprocessing_mask_annotation'
model_path = 'InceptionV3' #ResNet50 #VGG-16
clustering_path = 'Clustering'
plot_path = 'plots/Clustering' 
class_path = 'two_classes/masks' #celltypes

In [3]:
df_dataset = read_pickle(path=os.path.join(base_path, preprocessing_path, model_path, f"{model_path}_celltypes_lbl_df.pkl"))

In [4]:
print(df_dataset.head())

   Pseudonym  tile_id    lbl_mask  Acinar cells  Alpha cells  B cells  Basal  \
0  IAA2LDX17  (0, 10)  non-cancer           0.0          0.0      0.0    0.0   
1  IAA2LDX17  (0, 11)  non-cancer           0.0          0.0      0.0    0.0   
2  IAA2LDX17  (0, 12)  non-cancer           1.0          0.0      4.0    6.0   
3  IAA2LDX17  (0, 13)  non-cancer           0.0          0.0      0.0    0.0   
4  1C73PUTH4   (1, 2)  non-cancer           0.0          0.0      0.0    1.0   

   Beta cells  Classical_CEACAM  Classical_KRT7  ...  NK cells  Schwann cells  \
0         0.0               1.0             2.0  ...       0.0            0.0   
1         0.0               3.0             0.0  ...       0.0            0.0   
2         1.0              30.0             1.0  ...       2.0            2.0   
3         0.0               1.0             0.0  ...       0.0            0.0   
4         0.0               0.0             0.0  ...       0.0            0.0   

   T cells  iCAF  myCAF_ACTA2  m

In [7]:
X = df_dataset["Features"].to_list()
y = df_dataset["lbl_masks"] #lbl

In [8]:
y_dic = {idx: i for idx, i in enumerate(np.unique(y))}
classes = sorted(y_dic.items(), key=lambda item: item[0])
classes = [i[1] for i in classes]
print(y_dic)
print(classes)

{0: 'cancer', 1: 'non-cancer'}
['cancer', 'non-cancer']


In [6]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(y)

[0 0 0 ... 0 1 1]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=109)
print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({1: 7332, 0: 2268})
Testing target statistics: Counter({1: 1838, 0: 563})


In [8]:
def build_and_test(X_tr, X_te, y_tr, y_te, plot_dir:str, model_name:str, threshold=False, scaler:str='', balancer:str=''):
    print( model_name, scaler, balancer, threshold)

    direct = os.path.join(plot_dir, model_name, scaler, balancer, 'Threshold' if threshold else '') 
    os.makedirs(direct, exist_ok=True)

    #scale data
    if scaler == 'StandardScaler':
        std_scaler = StandardScaler().fit(X_tr + X_te)
        X_tr_scaled = std_scaler.transform(X_tr)
        X_te_scaled = std_scaler.transform(X_te)
    elif scaler == 'MinMaxScaler':
        minmax_scaler = MinMaxScaler().fit(X_tr + X_te)
        X_tr_scaled = minmax_scaler.transform(X_tr)
        X_te_scaled = minmax_scaler.transform(X_te)
    else:
        X_tr_scaled = X_tr
        X_te_scaled = X_te
    
    
    #balance dataset
    if balancer == 'RandomOversampler':
        over_sampler = RandomOverSampler(random_state=109)
        X_tr_sampled, y_tr_sampled = over_sampler.fit_resample(X_tr_scaled, y_tr)
    elif balancer == 'RandomUndersampler':
        under_sampler = RandomUnderSampler(random_state=109)
        X_tr_sampled, y_tr_sampled = under_sampler.fit_resample(X_tr_scaled, y_tr)
    else:
        X_tr_sampled, y_tr_sampled = X_tr_scaled, y_tr
    
    
    #build model
    if model_name == 'SVM':
        model = svm.SVC(kernel="linear", C=1.0, probability=True)
        model.fit(X_tr_sampled, y_tr_sampled)
        y_pred = model.predict(X_te_scaled)
    elif model_name == 'LogisticRegression':
        model = LogisticRegression(max_iter=1500)
        model.fit(X_tr_sampled, y_tr_sampled)
        y_pred = model.predict(X_te_scaled)
    else:
        print(f'WARNING no {model} model found!')
        
    #print metrices    
    result_dic = {f'Model': model,
                  f'Balancer': balancer,
                  f'Scaler': scaler,
                  f'Precision score': precision_score(y_te, y_pred),
                  f'Recall score':  recall_score(y_te, y_pred),
                  f'F1-score score': f1_score(y_te, y_pred),
                  f'Accuracy score': accuracy_score(y_te, y_pred),
                  f'Area under the ROC curve (AUC)': roc_auc_score(y_te, y_pred)}

    
    y_score = model.predict_proba(X_te)
    fpr0, tpr0, thresholds = roc_curve(y_te, y_score[:, 1])
    roc_auc0 = auc(fpr0, tpr0)
    
    # Calculate the best threshold
    best_threshold = None
    if threshold:
        J = tpr0 - fpr0
        ix = argmax(J) # take the value which maximizes the J variable
        best_threshold = thresholds[ix]
        # adjust score according to threshold.
        y_score = np.array([[1, y[1]] if y[0] >= best_threshold else [0, y[1]] for y in y_score])
        
    
    result_dic['cm'] = confusion_matrix(y_te, y_pred)
    
    
       
    # Plot metrics 
    plot_roc(y_te, y_score)
    plt.title(f'ROC curve of \n model:{model} scaler:{scaler} \n balancer:{balancer}')
    plt.tight_layout()
    plt.savefig(os.path.join(direct, 'roc.png'))
    plt.clf()
    plt.close()

    
    plot_precision_recall(y_te, y_score)
    plt.title(f'Precision Recall curve of \n model:{model} scaler:{scaler} \n balancer:{balancer}')
    plt.tight_layout()
    plt.savefig(os.path.join(direct, 'precision_recall.png'))
    plt.clf()
    plt.close()

    
    plot_confusion_matrix(y_te, y_pred, cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix of \n model:{model} scaler:{scaler} \n balancer:{balancer}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(os.path.join(direct, 'Confusion_Matrix.png'))
    plt.clf()
    plt.close()
    
    # Print a classification report
    result_dic['classification_report'] = classification_report(y_te, y_pred)
                
    return result_dic

In [ ]:
balancer_list = ['RandomOversampler', 'RandomUndersampler', '']
scaler_list = ['MinMaxScaler', 'StandardScaler', '']
ml_model_list = ['SVM', 'LogisticRegression']
threshold_list = [True, False]

classification_dic  = [build_and_test(X_tr=X_train, X_te=X_test, y_tr=y_train, y_te=y_test, plot_dir=os.path.join(base_path, plot_path, class_path, model_path), model_name = m, threshold=t, scaler=s, balancer=b)
                       for s in scaler_list
                       for b in balancer_list
                       for m in ml_model_list
                       for t in threshold_list]

SVM MinMaxScaler RandomOversampler True
SVM MinMaxScaler RandomOversampler False
LogisticRegression MinMaxScaler RandomOversampler True
LogisticRegression MinMaxScaler RandomOversampler False
SVM MinMaxScaler RandomUndersampler True
SVM MinMaxScaler RandomUndersampler False
LogisticRegression MinMaxScaler RandomUndersampler True
LogisticRegression MinMaxScaler RandomUndersampler False
SVM MinMaxScaler  True
SVM MinMaxScaler  False
LogisticRegression MinMaxScaler  True
LogisticRegression MinMaxScaler  False
SVM StandardScaler RandomOversampler True


In [ ]:
df_classification = pd.DataFrame(classification_dic)
df_classification

In [ ]:
write_pickle(path= os.path.join(base_path, clustering_path, class_path, model_path, f"{model_path}_classification_dict.pkl"), obj=classification_dic)
write_pickle(path= os.path.join(base_path, clustering_path, class_path, model_path, f"{model_path}_classification_df.pkl"), obj=df_classification)